## Cooking with ClarityNLP - Session #10: Clinical Trials and Language Models

For today's cooking session we will use ClarityNLP to find mentions of chemotherapy regimens in clinical trial eligibility criteria. We will then use the extracted text to build an ngram language model that recognizes usage contexts for chemotherapy regimens. We will also estimate the performance of our model via K-fold cross validation.

For details on installing and using ClarityNLP, please see our [documentation](https://claritynlp.readthedocs.io/en/latest/index.html).  We welcome questions via Slack or on [GitHub](https://github.com/ClarityNLP/ClarityNLP/issues).

## ClinicalTrials.gov and the AACT Database

The U.S. Library of Medicine maintains a website called [ClinicalTrials.gov](https://clinicaltrials.gov/), which hosts a large database of clinical trial information. The data includes the trial objectives, inclusion and exclusion criteria, the participating organizations, the personnel conducting the trial, and other related information. Each clinical trial is assigned a unique **NCT ID**, such as NCT03601923, which happens to be the ID for a now-recruiting trial to evaluate the drug Niraparib as a treatment for pancreatic cancer.

The home page for ClinicalTrials.gov provides a web form by which users can enter search criteria and find clinical trials of interest:

![clinicaltrials_gov.png](session_10/clinicaltrials_gov.png)

This form is convenient for interactive searches, but larger-scale analysis of the clinical trial data requires access to the database, which this site does not provide.

Luckily, there is a [related AACT website](https://aact.ctti-clinicaltrials.org/) (AACT == Aggregate Analysis of ClinicalTrials.gov) that hosts a downloadable PostgreSQL database containing all of the data at clinicaltrials.gov. As of this writing, the current download is 871 MB in size, and it contains data for more than 289,000 clinical trials. The download file is updated at near-daily intervals. If you want to run natural language processing or machine learning algorithms on the clinical trial data, the AACT database is what you need. The AACT website contains links to download and installation instructions, the data dictionary and database schema, and lots of other documentation.

### Loading the AACT Database into Solr

To be able to interrogate the AACT data with ClarityNLP, the relevant text from the clinical trials needs to be ingested into Solr. At GA Tech we downloaded a snapshot of the AACT database, installed it, got it working, and ingested the eligibility criteria into our Solr instance. The eligibility criteria can be found in the ``criteria`` column of the ``eligibilities`` table.

Some python scripts that we used can be found in our [utilities project](https://github.com/ClarityNLP/Utilities). Look at the .py files containing *aact* in the filename if you're interested. Documentation on the fields that ClarityNLP expects can be found in our [documentation](https://claritynlp.readthedocs.io/en/latest/developer_guide/technical_background/solr.html).

Our data ingestion process maps these [required fields](https://claritynlp.readthedocs.io/en/latest/developer_guide/technical_background/solr.html) as follows:
* ``report_type``: either ``"Clinical Trial Inclusion Criteria"`` or ``"Clinical Trial Exclusion Criteria"``
* ``report_text``: text of the inclusion or exclusion criteria
* ``source: AACT``
* ``subject``: the NCT ID of the trial

These field mappings will be important for understanding the NLPQL presented below.

**IMPORTANT NOTE**: After ingesting this data we found that our python scripts did not capture all of the eligibility criteria for some trials. The text of the eligibility criteria unfortunately does not conform to the official data dictionary in all instances. The data dictionary [states the following](https://prsinfo.clinicaltrials.gov/definitions.html#EligibilityCriteria) for the eligibility criteria:

<pre>
Eligibility Criteria *
Definition: A limited list of criteria for selection of participants in the clinical study,
provided in terms of inclusion and exclusion criteria and suitable for assisting potential
participants in identifying clinical studies of interest. Use a bulleted list for each
criterion below the headers "Inclusion Criteria" and "Exclusion Criteria".
Limit: 15,000 characters. 
</pre>

Note the bulleted list requirement. Some trials do not use a bulleted list for the criteria; others are missing either inclusion or exclusion criteria or both; others alternate back and forth between inclusion and exclusion, etc. Our ingest scripts do not capture all of the data under these circumstances. We estimate that approximately 5% of the eligibility criteria are non-conformant.

We are working on improved data ingest code and will update our scripts when we are satisfied that we can cleanly capture all of the criteria. For purposes of this cooking session, though, the scripts work well enough.

## Chemotherapy Regimens

Our goal is to be able to find mentions of chemotherapy regimens in the clinical trial eligibility criteria. We would also like to be able to assign a score to each criterion that indicates how confident we are in the identification. For some regimens such as BEACOPP, FOLFOX, ProMACE-CytaBOM, and XELOX this is not too difficult. For other regimens such as CA, ICE, and BEAM this task is somewhat more difficult, since the regimen names match those of common words or abbreviations.

Our approach to this problem is to use ClarityNLP to extract regimen names and the surrounding text from the clinical trial eligibility criteria. We then normalize the text, replace the regimen names with a token, and compute all ngrams that include the regimen token. These ngrams provide us with sample contexts for the use of regimen names. By counting all such ngrams and scoring them in a smoothed language model, we can assign a probability to each ngram and use it to estimate the likelihood that a given ngram contains a regimen name.

### Regimen Names

So where can one find a list of the names of chemotherapy regimens? There happens to be a comprehensive, peer-reviewed oncology wiki at [HemeOnc.org](https://hemonc.org/wiki/Main_Page) that contains information on treatment regimens for many different types of cancer. One of our intrepid co-workers scraped this wiki and generated hundreds of NLPQL files for extracting the treatment regimens and relevant drugs from clinical text. The full list can be found [here](https://github.com/ClarityNLP/Utilities/tree/master/regimen_nlpql), along with NLPQL for related drugs and conditions. A JSON file containing all of this data can be found [here](https://github.com/ClarityNLP/Utilities/blob/master/cancer/regimen_tree.json).

We provide a python utility script called ``get_all_regimens.py`` that extracts the regimen names from the JSON data and writes them to a .csv file. You can find this utility script [here](https://github.com/ClarityNLP/Utilities/blob/master/get_all_regimens.py).  Running this script generates the file ``all_regimen_names.csv``, which you can find in our github repo in the support folder for this notebook: ``ClarityNLP/notebooks/cooking/session_10/``. Here is a sample of what the file contains (**1961 lines total**):

![all_regimens.png](session_10/all_regimens.png)

The regimen name(s) are found in the first column. Alternate names are found in the second column.

Believe it or not, there are still other chemotherapy regimen names not contained in this file. We have collected additional regimen names (mostly from Wikipedia and clinical trial data) and saved them to the file ``notebooks/session_10/supplemental_chemo_regimens.txt``. This file contains 88 additional regimens.

### NLPQL for Finding Chemotherapy Regimens

If we want to use ClarityNLP to find chemotherapy regimens in clinical text, we need to generate an NLPQL file containing the relevant instructions. Here is a template for an NLPQL file that we can use to find chemotherapy regimens in the AACT data:

<pre>
phenotype "Chemotherapy Regimens" version "1";
include ClarityCore version "1.0" called Clarity;

// NOTE: uses field mappings described above
documentset Docs:
    Clarity.createDocumentSet({
        "report_types":[
            "Clinical Trial Inclusion Criteria",
            "Clinical Trial Exclusion Criteria"
        ],
        "filter_query":"source:AACT"
    });

// << INSERT TERMSET FOR CHEMOTHERAPY REGIMEN NAMES HERE >>

// find ANY mention of an entry in our termset
define hasRegimenTerm:
    Clarity.TermFinder({
        termset: [RegimenTerms],
        documentset: [Docs]
    });
</pre>

This NLPQL file does the following things:

* Creates a document set customized for our AACT data
* Creates a placeholder for a chemotherapy regimen termset
* Creates a [TermFinder](https://claritynlp.readthedocs.io/en/latest/api_reference/nlpql/term_finder.html) task to search for the regimen terms in the AACT data

It is straightforward to extract the data from the CSV file and the supplemental regimen file and generate an NLPQL termset containing all of the ~2000 regimen names. Unfortunately, the number of false positives that ClarityNLP finds is overwhelming. This is because many of the regimen names such as HAM, RICE, ACT, and others are common English words. Most two and three-letter regimen acronyms seem to collide with other common abbreviations in this data.

There is also another problem with such a large termset. **Solr has a limit on the number of Booleans that it can use in a given query.** This value can be found in the ``solrconfig.xml`` file as the entry ``maxBooleanClauses``. The default seems to be 1024. ClarityNLP converts the termset into a Boolean query, so running with a huge termset may cause you to exceed your system's ``maxBooleanClauses`` limit. Please check the value in your ``solrconfig.xml`` file and adjust if necessary.

To overcome both of these problems, we have ruthlessly **pruned the termlist** to remove drug names, common acronyms, and any regimen names that are not present in our AACT data. This process required trial-and-error experimentation and repeated runs to produce a result amenable to the **automated** analysis described below. Our final pruned termset can be found in ``notebooks/cooking/session_10/regimen_termset.txt``. This termset includes **349** regimen names.

With the termset in hand, you can paste it into the NLPQL file above and run it. The full NLPQL file can be found in ``notebooks/cooking/session_10/find_regimens.nlpql``.

When we run this file on our AACT dataset it generates a .csv result file containing 2077 entries. This file has been saved to ``notebooks/cooking/session_10/phenotype_regimens.csv``.

### Dataset Generation

We would now like to use the results from ClarityNLP to generate an ngram language model. Unfortunately, we cannot just extract the [sentence](https://claritynlp.readthedocs.io/en/latest/api_reference/nlpql/term_finder.html) field from the results, since **the sentence tokenizer is often confused by the strange formatting of the clinical trial criteria data**. We need to do some more work to extract the sentences that we need.

First some preliminaries:

In [1]:
import re
import os
import sys
import csv
import nltk
import random
import statistics
import subprocess

# max n for ngrams
MAX_N = 5

# output file for language model cross-validation data
SENTENCE_FILE = 'session_10/sentences.txt'

# output file for sentences that get discarded
DISCARD_FILE = 'session_10/discard_sentences.txt'

# error log
LOG_FILE = 'session_10/error_log.txt'

# replace all chemotherapy regimen names with this token
REGIMEN_TOKEN = '<regimen>'

# these tokens denote the start and end of a sentence in the ngram model
START_TOKEN = '<s>'
END_TOKEN   = '</s>'

# name of the final language model file
FINAL_MODEL_FILE = 'session_10/final_model.arpa'

We will need a function to load the phenotype .csv file and extract the "sentence" and matching term from the termset. We will also need a function to load the termset, since the terms need to undergo the same text normalization process as the sentences. A function to load a list of drug names is also needed.

Here is the loading code:

In [2]:
###############################################################################
def load_csv_data(csv_file):
    """
    Extract the 'sentence' field from the file and return list of strings.
    """

    data = []
    with open(csv_file, 'rt') as infile:
        dict_reader = csv.DictReader(infile)
        for row_dict in dict_reader:
            sentence = row_dict['sentence']
            data.append(sentence)

    return data


###############################################################################
def load_termset(termset_file):
    """
    Extract individual regimen terms from an NLPQL termset.
    """

    term_list = []
    with open(termset_file, 'rt') as infile:
        for line in infile:
            line = line.strip()

            # regimen names are quoted
            if not line.startswith('"'):
                continue

            # strip quotes and commas
            if line.endswith('",'):
                regimen = line[1:-2]
            else:
                # final term
                regimen = line[1:-1]

            term_list.append(regimen)
            
    return term_list

###############################################################################
def load_drugs(drug_file):
    """
    Load a list of drug names from the given file. Assumes a file format of
    one string per line. Returns a set for faster lookup.
    """
    
    drug_set = set()
    with open(drug_file, 'rt') as infile:
        for line in infile:
            if line.isspace() or 0 == len(line) or line.startswith('#'):
                continue
            drug_set.add(line.strip())
            
    return drug_set

We will normalize the text by converting it to lowercase, stripping punctuation, removing extraneous Unicode codepoints, and a few other transformations. Here is the code to cleanup the text:

In [3]:
###############################################################################
def cleanup_text(sentences):
    """
    Normalize the input data by performing a series of cleanup operations.
    The input is assumed to be a list of strings. Returns a list of strings
    containing the cleaned data. In this function a 'sentence' could also be
    a regimen name, since this function is used to clean both the sentence and
    regimen data.
    """

    cleaned_sentences = []
    for sent in sentences:

        # convert to lowercase
        sent = sent.lower()

        # strip registered trademark and related symbols
        # Gliadel wafer sometimes has these
        sent = re.sub(r'(\u00AE|\u2122|\u2120|\u00A9|\u2117)', '', sent)
        
        # replace 'r/r' with 'relapsed or refractory'
        sent = re.sub(r'\br/r\b', 'relapsed or refractory', sent)

        # replace symbols used in regimen names with whitespace
        sent = re.sub(r'[-+/&]', ' ', sent)

        # replace i.e. with ie, .e.g. with eg, to prevent single-letter
        # tokens when punctuation is replaced with whitespace
        sent = re.sub(r'\bi\.e\.?\b', 'ie', sent)
        sent = re.sub(r'\be\.g\.?\b', 'eg', sent)
        
        # replace punctuation with whitespace
        sent = re.sub(r'[^a-z0-9]', ' ', sent)

        # correct misspellings:
        #    'regiment'         => regimen
        #    'hypersensibility' => 'hypersensitivity'
        sent = re.sub(r'\bregiment\b', 'regimen', sent)
        sent = re.sub(r'\bhypersensibility\b', 'hypersensitivity', sent)
        
        # collapse repeated spaces to a single space
        sent = re.sub(r'\s+', ' ', sent)

        cleaned_sentences.append(sent.strip())

    return cleaned_sentences


To normalize the regimen terms, we normalize the regimen text and then enter a sufficient number of variants to capture common usage. For instance, a regimen such as "XELOX and Bevacizumab" could appear in the clinical trial eligibility criteria in any of these forms:

<pre>
XELOX & Bevacizumab
XELOX / Bevacizumab
XELOX + Bevacizumab
    
XELOX and Bevacizumab
XELOX plus Bevacizumab
    
XELOX&Bevacizumab
XELOX/Bevacizumab
XELOX+Bevacizumab
</pre>

We can match all of these forms after the normalization process with three variants:

<pre>
xelox and bevacizumab
xelox plus bevacizumab
xelox bevacizumab
</pre>


In [4]:
###############################################################################
def cleanup_terms(term_list):
    """
    Apply text cleanup operations to the list of regimen names. Also enter
    additional variants to capture the inconsistent usage of '&', +', and '/'
    in the clinical trial data.
    """
    
    regimens = cleanup_text(term_list)

    # enter variants for "word1 + word2" forms
    result_set = set()
    for regimen in regimens:

        words = regimen.split()
        if 1 == len(words):
            result_set.add(regimen)
        else:
            if 3 == len(words) and 'and' in words:
                index = words.index('and')
                result_set.add(regimen)
                words[index] = 'plus'
                result_set.add(' '.join(words))
                del words[index]
                result_set.add(' '.join(words))
            else:
                result_set.add(regimen)

    results = sorted(list(result_set))
    return results


The sentence normalization process applies the same text cleanup function, then substitutes **``<regimen>``** for each regimen term contained in the sentence. This is referred to in the code as the **REGIMEN_TOKEN**. This code also checks the common CHOP regimen for matches against "Childrens Hospital of Philadelphia", which unfortunately shares the same acronym.

In [5]:
###############################################################################
def cleanup_sentences(sentence_list, term_list):
    """
    Apply cleanup operations to the sentence list. Substitute the REGIMEN_TOKEN
    for all identified regimen terms in the sentence. Do some special handling
    for the CHOP regimen to remove sentences referring to the Children's
    Hospital of Philadelphia, which shares the same acronym.
    """
    
    results = set()
    sentence_list = cleanup_text(sentence_list)
    
    # substitute <regimen> for all identified regimen names

    counter = 0
    for i in range(len(sentence_list)):
        before = sentence_list[i]

        after = before
        for term in term_list:
            after = re.sub(r'\b' + term + r'\b', REGIMEN_TOKEN, after)
        if before != after:
            # this sentence contains a regimen name
            # ignore CHOP == Children's Hospital of Philadephia
            # picu == Pediatric Intensive Care Unit
            if -1 != after.find('childrens') or    \
               -1 != after.find('philadelphia') or \
               -1 != after.find('picu') or         \
               -1 != after.find('travel to'):
                continue
            else:
                results.add(after)

        counter += 1
        if 0 == counter % 1000:
            print('\tprocessed {0}/{1}'.format(counter, len(sentence_list)))
            
    return list(results)


The text extracted by ClarityNLP can be resolved into individual sentences with the following code. The "bullet" character in the clinical trial data seems to be the "dash" character, which can appear either as an isolated dash or a dash immediately following the period of the preceding sentence. This function finds such occurrences and splits the text at those points. It also removes the "Inclusion Criteria" and "Exclusion Criteria" headers, should they be present.

In [6]:
###############################################################################
def resolve_sentences(text):
    """
    Extract individual sentences from the text and return as a list.
    Ignore the header lines and keep only the text relevant to the
    clinical trial. A 'sentence' may actually only be a phrase or a brief
    description in a bulleted list.
    """

    sentence_list = []
    
    # replace occurrences of '.- ', which denote the end of a sentence
    # followed by a new bulleted item with '. - ' to isolate the hyphen
    text = re.sub(r'\.\-\s', '. - ', text)
    
    # split on ' - ' to isolate the bulleted items
    chunks = text.split(' - ')

    # run the sentence tokenizer on the chunks
    for chunk in chunks:
        candidates = nltk.sent_tokenize(chunk)

        # remove sentences consisting of '(In|Ex)clusion Criteria:'
        for sent in candidates:
            match = re.match(r'(In|Ex)clusion\s+Criteria:\s*\Z', sent, re.IGNORECASE)
            if match:
                continue
            sentence_list.append(sent)

    return sentence_list


Finally, here is the code that generates data for the ngram language model. We use a keyword search to filter the text found by ClarityNLP. A candidate sentence is kept if any of these cancer-related keywords appear in the sentence. We also keep a sentence if it contains a mention of a cancer drug, immunotherapy drug, or a drug used treat Graft vs Host disease.

These lookups are our substitute for a laborious human-in-the-loop review of thousands of sentences. The drawback of doing this is that our results may, to some extent, be trained on usages common only to sentences that include these terms.

In [7]:
###############################################################################
def gen_language_model_data(csv_file, termset_file, drug_file):
    """
    Load a ClarityNLP phenotype .csv file and extract the 'sentence' and 'term'
    fields. The 'sentence' field can contain multiple sentences, so resolve
    the text into individual sentences. Write sentences and terms to disk.
    """

    # Keep the text extracted by ClarityNLP if any of these terms are present,
    # since they are all relevant to cancer, chemotherapy, and immunotherapy.
    # This is our (somewhat inadequate) substitute for human-in-the-loop
    # review of the sentences.
    # The 'r <regimen>' and '<regimen> r' options catch R-REGIMEN or REGIMEN-R
    # names that we might have missed (R means 'rituximab').
    str_keywords = r'(adjuvant|(b|t) cell|antiemetic|cancer|carcinoma|chemo|'             +\
                   r'chemoimmunotherapy|chemotheraputic|chemotherapy|cml|course(s)? of|'  +\
                   r'combination therapy|\bcr\b|cycle|(first|second) line|immunotherapy|' +\
                   r'induction|leukemia|lymphoma|malignancy|malignant|oncologist|'        +\
                   r'oncologic|oncology|metastases|monotherapy|refractory|remission|'     +\
                   r'relapse|resection|response|round|stage|stem cell|treatment|'         +\
                   r'toxic|toxicity|tumor|upfront|'                                       +\
                   r'r <regimen>|<regimen> r|'                                            +\
                   r'(?<!<)regimen(?!>))'
    regex_keywords = re.compile(str_keywords, re.IGNORECASE)
    
    # data is a list of (term, text) tuples
    data = load_csv_data(csv_file)

    # load the NLPQL termset
    term_list = load_termset(termset_file)
    
    # get the drug list as a set of strings, for faster lookup
    drug_set = load_drugs(drug_file)
    
    sentence_list = []
    for text in data:
        new_sentences = resolve_sentences(text)
        sentence_list.extend(new_sentences)
        
    # cleanup terms and sentences, substitute <regimen> token
    print('cleaning terms...')
    term_list = cleanup_terms(term_list)

    print('cleaning sentences...')
    sentence_list = cleanup_sentences(sentence_list, term_list)
    
    # try for a keyword or drug match; if no match, discard the sentence
    keep_sentences    = []
    discard_sentences = []
    for s in sentence_list:
        
        num_kept = len(keep_sentences)
        match = regex_keywords.search(s)
        if match:
            keep_sentences.append(s)
        else:
            # look for a known cancer-related drug name
            words = s.split()
            for w in words:
                if w in drug_set:
                    keep_sentences.append(s)
                    break
                    
        if len(keep_sentences) == num_kept:
            # discard this sentence
            discard_sentences.append(s)
    
    # write the keep/discard sentences to separate files
    data = [
        (SENTENCE_FILE, keep_sentences),
        (DISCARD_FILE, discard_sentences)
    ]
    
    for data_file, data_list in data:
        with open(data_file, 'wt') as outfile:
            for sentence in data_list:
                outfile.write('{0}\n'.format(sentence))
            print('\nWrote {0} sentences to "{1}".'.format(len(data_list), data_file))

## Ngram Language Model and K-Fold Cross-Validation Code

Now that we have the code to generate the cross validation data, we turn our attention to the language model.

An ngram language model is essentially a set of ngrams and their scores. The score represents the probability of occurrence for the ngram in the training text. Ngram frequency counts are used as a starting point for the probability estimate, but an additional "smoothing" procedure must be applied to make the model useful. You can find more information about ngram models in chapter three of the latest draft of [Jurafsky and Martin](https://web.stanford.edu/~jurafsky/slp3/). A comprehensive overview of smoothing techniques can be found in a [technical report by Chen and Goodman](https://dash.harvard.edu/handle/1/25104739).

To generate the language model, we will use the state-of-the art in language model generation software, [KenLM](https://kheafield.com/code/kenlm/). This code is lightning fast and can handle language data at web scale. It is written in C++ for performance, so a binary will need to be built on your system if you want to run it.

Download and build the code for your system with the following commands. You will need to install ``boost``, ``Eigen``, ``cmake``, and ``wget`` if you do not already have them installed on your system (or you can just download the tarball directly without using wget):

<pre>
wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
mkdir kenlm/build
cd kenlm/build
cmake ..
make -j2
</pre>

The binary that we need is ``kenlm/build/lmplz``. A binary built for a MacOS High Serria system (MacOS 10.13.6) can be found in ``notebooks/cooking/session_10/mac_lmplz``.

There is a python interface for kenlm, but the language model generation features do not seem to be callable from python.

Our plan of attack will be as follows. First, we will randomly scramble our regimen-containing sentence dataset. Then we will divide it into K subsets, using one of the subsets as the test data and the remainder as the training data. We will use kenlm to generate a language model from the training data, then evaluate it on the test data. The evaluation process will report a score when it concludes. We will repeat the evaluation process K times and conclude by generating a final model trained on all of the data. The results of the evaluation will be the average and standard deviation of all scores.

The kenlm code wants to read the training data from a file on disk, so we need a function to write a list of sentences to a file:

In [8]:
###############################################################################
def write_training_file(k, training_sentences):
    """
    Write a list of training sentences to disk, one per line.
    Returns the name of the file.
    """
    
    filename = 'session_10/train_data_{0}.txt'.format(k)
    with open(filename, 'wt') as outfile:
        for s in training_sentences:
            outfile.write('{0}\n'.format(s))

    return filename


We will also need a function to load our training and test sentences from a file:

In [9]:
###############################################################################
def load_sentences(filepath):
    """
    Load sentences from a file, assumed to be one per line. Returns the data
    as a list of strings.
    """

    sentences = set()
    with open(filepath, 'rt') as infile:
        for line in infile:
            sentence = line.strip()
            assert REGIMEN_TOKEN in sentence
            sentences.add(sentence)
            
    return list(sentences)


We also need code to generate ngrams confined to a window near the REGIMEN_TOKEN. The start and end of sentence will be denoted with the START_TOKEN and END_TOKEN strings defined in the initial code cell above.

In [10]:
###############################################################################
def gen_ngrams(n, sentence):
    """
    Generate all ngrams of length n from the sentence that include
    the REGIMEN_TOKEN. Returns a list of these ngrams. Denote the start and
    end of the sentence with START_TOKEN and END_TOKEN.
    """
    
    words = sentence.split()
    m = len(words)
    assert REGIMEN_TOKEN in words

    ngram_list = []
    i = words.index(REGIMEN_TOKEN)
    for start in range(i-n+1, i+1):
        if start < -1:
            # no ngram, too far left
            continue
        end = start + n
        if end > m:
            # no ngram, too far right
            continue
        ngram = []
        for j in range(start, end):
            if j < 0:
                ngram.append(START_TOKEN)
            elif j >= m:
                ngram.append(END_TOKEN)
            else:
                ngram.append(words[j])

        assert n == len(ngram)
        ngram_list.append(ngram)

    return ngram_list


The kenlm code generates a language model file in .arpa format. This is a simple format [documented here](http://www.speech.sri.com/projects/srilm/manpages/ngram-format.5.html). Here is a function to read an .arpa file and extract the ngrams and their probabilities:

In [11]:
###############################################################################
def load_arpa_file(filepath):
    """
    Load a language model file in .arpa format. Build a dict that maps the
    ngram length n to a list of (probability, ngram) tuples. Sort the list
    for each n in decreasing order of probability and return the dict.
    """

    section = 0
    counts = {}
    ngram_dict = {}
    with open(filepath, 'rt') as infile:
        for line in infile:
            if line.isspace() or 0 == len(line):
                continue
            if line.startswith('\end'):
                section = -1
                continue

            # match \1-grams, \2-grams, etc.
            match = re.match(r'\\(?P<num>\d+)\-grams', line)
            if match:
                section = int(match.group('num'))
                assert section >= 1
                ngram_dict[section] = []
                continue

            if section == 0:
                # get ngram counts
                match = re.match(r'\Angram\s(?P<n>\d+)=(?P<count>\d+)', line)
                if match:
                    n = int(match.group('n'))
                    count = int(match.group('count'))
                    counts[n] = count
            else:
                components = line.split()
                prob = float(components[0])
                ngram = components[1:1+section]
                ngram_dict[section].append( (prob, ngram) )
    
    # sort the ngrams for each n value in order of decreasing score
    for n, tup_list in ngram_dict.items():
        tup_list = sorted(tup_list, key=lambda x: x[0], reverse=True)
        ngram_dict[n] = tup_list

    return ngram_dict


The kenlm language models include all ngrams generated from the training text. We only want those ngrams that contain the REGIMEN_TOKEN, so this function filters out the extraneous ngrams:

In [12]:
###############################################################################
def filter_ngrams(ngram_dict):
    """
    The ngram_dict is a dict that maps the ngram length n to a list of 
    (probability, ngram) tuples. Remove all ngrams from the dict that do not
    include the REGIMEN_TOKEN.
    """
    
    for n, tup_list in ngram_dict.items():
        keep_list = []
        for prob, ng in tup_list:
            if REGIMEN_TOKEN in ng:
                keep_list.append( (prob, ng) )
        ngram_dict[n] = keep_list

Since the kenlm language model must be generated by running the binary file that we built above, here is a function that launches a shell to run the binary and generate the model file:

In [13]:
###############################################################################
def gen_language_model(train_file, model_file, log_file):
    """
    Spawn a shell and run the kenlm binary. If the run fails (probably because
    of insufficient data), write the error message to the log file. Returns a
    Boolean indicating success or failure.
    """
    
    # shell command to run kenlm
    command = 'session_10/mac_lmplz -o 5 --discount_fallback < {0} > {1}'.format(train_file, 
                                                                                 model_file)

    # run the shell command
    result = subprocess.run(command, shell=True, stderr=subprocess.PIPE)
    if 0 != result.returncode:
        log_file.write('\nkenlm run failure: ')
        log_file.write(result.stderr)
        log_file.write('\n')
        return False
    
    return True


This is our model evaluation code. It works by finding all ngrams in the test and training data that include the ``REGIMEN_TOKEN``. For each such ngram in the test data, it tries to find a matching ngram in the training data. It checks all values of n from 3 to MAX_N (currently 5). It does not check bigrams since they tend to not be very informative. If multiple ngram matches are found, the winner is the ngram with the highest probability. If no matches are found it declares failure for that test ngram. The success percentage is returned as the score for the evaluation process.

The idea here is that if the training data adequately represents the forms of expression used for chemotherapy regimens in clinical trial data, then the scores should be closer to 1 than to 0. 

In [14]:
###############################################################################
def best_candidate(ngram_dict, sentence):
    """
    Generate all REGIMEN_TOKEN-containing ngrams of lengths 3 .. MAX_N from
    the sentence. Start with the sentence ngrams of length n = MAX_N. 
    For each ngram of this length, look for a match in the training ngrams. 
    If a match is found, save it if its probability is higher than that found
    thus far. Then repeat with n = MAX_N - 1.
    
    Returns a tuple of the highest-probability matching ngram, if any, along
    with the value of n and the probability. If no ngram is found, the first
    component of the returned tuple has the value None.
    """
    
    best_n = None
    best_prob = -9999999.0
    best_ngram = None
    
    # skip bigrams in the ngram search, not necessarily informative
    for n in reversed(range(3, MAX_N+1)):

        # get the training ngrams that include "<regimen>"
        #     this is a list of (prob, ngram) tuples
        #     the ngram itself is a list of strings
        train_ngrams = ngram_dict[n]

        # list of ngrams from test sentence s that include "<regimen>"
        test_ngrams = gen_ngrams(n, sentence)

        for ng_test in test_ngrams:
            # search for it in the training ngrams
            for prob, ng_train in train_ngrams:
                if ng_test == ng_train:
                    # test ngram matches the training ngram
                    # keep it if has higher probability
                    if prob > best_prob:
                        best_n = n
                        best_prob = prob
                        best_ngram = ng_test
                        
    return (best_n, best_prob, best_ngram)

###############################################################################
def evaluate(ngram_dict, test_data):
    """
    Run the model on all test sentences. Accumulate a count of all matches
    found and return a score equal to the percentage of the test set for which
    a match was found.
    """

    success_count = 0
    failure_count = 0
    sentence_count = len(test_data)
    
    for s in test_data:
        
        best_n, best_prob, best_ngram = best_candidate(ngram_dict, s)
                            
        if best_n is not None:
            success_count += 1
        else:
            failure_count += 1

    assert success_count + failure_count == sentence_count

    score = success_count / sentence_count
    return score


Finally, here is the K-fold cross-validation code. The default value of K is 10.

In [15]:
###############################################################################
def cross_validate(filepath, K=10):
    """
    Perform K-fold cross validation of our model on the given data file.
    The data file contains preprocessed and cleaned sentences, one per line.
    Each sentence contains one or more instances of REGIMEN_TOKEN.
    """

    orig_sentence_list = load_sentences(filepath)
    m = len(orig_sentence_list)
    print('Loaded {0} unique sentences.'.format(m))

    # shuffle the sentences
    indices = [i for i in range(m)]
    random.shuffle(indices)
    sentence_list = [orig_sentence_list[indices[i]] for i in indices]

    # split data into K chunks
    chunksize = m // K
    print('K: {0}, chunksize: {1}.'.format(K, chunksize))

    # open the error log
    log_file = open(LOG_FILE, 'wt')
    
    scores = []
    
    # do the K-fold cross validation
    for k in range(K):

        print('*** Iteration {0}/{1} ***'.format(k+1, K))
        
        # index range for the kth chunk
        lo = int((k+0) * chunksize)
        hi = int((k+1) * chunksize)

        # the test data is the kth chunk
        test_data = sentence_list[lo:hi]

        # the training data is everything else
        train_data = sentence_list[:lo]
        train_data.extend(sentence_list[hi:])
        
        # consistency checks
        assert len(train_data) + len(test_data) == m
        for s in test_data:
            assert s not in train_data

        # write training sentences to a file, to be loaded by kenlm
        train_file = write_training_file(k, train_data)
        
        # name of kenlm .arpa model file
        model_file = 'session_10/model_{0}.arpa'.format(k)

        # run kenlm and create the .arpa file
        if not gen_language_model(train_file, model_file, log_file):
            print('kenlm run failed...check error log')
            print('skipping this iteration...')
            continue
        
        # load the language model file, get the ngrams with probabilities
        # map of n -> [ (prob_0, [words_0]), (prob_1, [words_1]), ...] 
        ngram_dict = load_arpa_file(model_file)

        # keep only those ngrams that contain the <regimen> token
        filter_ngrams(ngram_dict)

        # print stats on ngrams containing the <regimen> token
        print('\tNgrams containing REGIMEN_TOKEN: ')
        for n, tup_list in ngram_dict.items():
            if n > 1:
                print('\tn = {0}, count: {1}'.format(n, len(tup_list)))
        
        # evaluate and record score for this round
        score = evaluate(ngram_dict, test_data)
        print('\tscore: {0:0.4f}'.format(score))
        
        scores.append(score)

    print('\nMean score: {0:.4f}'.format(statistics.mean(scores)))
    print('  Std. dev: {0:.4f}'.format(statistics.stdev(scores)))

    # generate final model using all data
    gen_language_model(filepath, FINAL_MODEL_FILE, log_file)

Run the next cell to generate the language model data (``session_10/sentences.txt``):

In [16]:
phenotype_file = 'session_10/phenotype_regimens.csv'
termset_file   = 'session_10/regimen_termset.txt'
drug_file      = 'session_10/drug_names.txt'

# generate the data
gen_language_model_data(phenotype_file, termset_file, drug_file)

cleaning terms...
cleaning sentences...
	processed 1000/6891
	processed 2000/6891
	processed 3000/6891
	processed 4000/6891
	processed 5000/6891
	processed 6000/6891

Wrote 1266 sentences to "session_10/sentences.txt".

Wrote 212 sentences to "session_10/discard_sentences.txt".


Run the code in the next cell to generate, train, and test language models on the test data:

In [17]:
# perform the cross-validation
K = 10
cross_validate(SENTENCE_FILE, K)

Loaded 1266 unique sentences.
K: 10, chunksize: 126.
*** Iteration 1/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 490
	n = 3, count: 2193
	n = 4, count: 3906
	n = 5, count: 5084
	score: 0.8016
*** Iteration 2/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 484
	n = 3, count: 2185
	n = 4, count: 3880
	n = 5, count: 5064
	score: 0.7937
*** Iteration 3/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 482
	n = 3, count: 2182
	n = 4, count: 3881
	n = 5, count: 5062
	score: 0.7778
*** Iteration 4/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 480
	n = 3, count: 2157
	n = 4, count: 3866
	n = 5, count: 5043
	score: 0.7698
*** Iteration 5/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 492
	n = 3, count: 2195
	n = 4, count: 3887
	n = 5, count: 5070
	score: 0.8175
*** Iteration 6/10 ***
	Ngrams containing REGIMEN_TOKEN: 
	n = 2, count: 488
	n = 3, count: 2203
	n = 4, count: 3894
	n = 5, count: 5053
	score: 0.8889
*** Iteration 7/10 ***
	Ngrams cont

We observe that our model can correctly find mentions of chemotherapy regimens slightly more than 80% of the time.

**We contend that we can improve the accuracy of our model and move the score towards 1.0 by accumulating more training data.** With more data, the ability of the model to correctly recognize proper usage of chemotherapy regimens should increase.

## Model Evaluation

We can assess the model by running it on the sentences that were discarded. We would expect the score for these sentences to be substantially less the 80% score we obtained on the training sentences. The score will not be zero for the following reasons:

* Our pruning heuristics above are not 100% reliable, and we incorrectly discard some valid data
* Our pruning heuristics may generate a data set subtly dependent on our keywords
* Sometimes an ngram from the training data occurs in the discarded sentences

The code in the next cell performs this evaluation:

In [18]:
# load the model trained on all the data
ngram_dict = load_arpa_file('session_10/final_model.arpa')

sentences = []
with open(DISCARD_FILE, 'rt') as infile:
    for line in infile:
        sentences.append(line.strip())
        
print('Evaluating the final model on discarded sentences...')
score = evaluate(ngram_dict, sentences)
print('\nScore on discarded sentences: {0:.4f}'.format(score))

Evaluating the final model on discarded sentences...

Score on discarded sentences: 0.4764


### Conclusions

This score on the discarded data is substantially less than the ~80% score that we obtained on our training data, confirming that the model does indeed exhibit worse performance on the discarded sentences. This is what we had hoped for. Hence our smoothed ngram language model does have some predictive ability, notwithstanding the fact that it was trained on a small corpus of only 1266 sentences.

The score on the discarded data probably has a lower bound that we will never be able to overcome, since the intersection of the following sets is likely to be nonempty:

* the set of ngrams used to state chemotherapy regimens in clinical trial eligibility criteria
* the set of ngrams used for common words and abbreviations that also happen to be chemotherapy regimen names

Thus this ngram language model can only be one component of a still-to-be-determined algorithm for **reliably** identifying chemotherapy regimen names in clinical trial data. We plan to improve this model by incorporating machine learning techniques, perhaps involving word embeddings or other context-aware methods.

We may revisit this topic in future cooking sessions. Stay tuned!